In [1]:
import torch
import torch.nn as nn

C:\Users\radcl\AppData\Roaming\Python\Python313\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [6]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [7]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [8]:
input = torch.randn( 3, 4, 768 )

In [9]:
input

tensor([[[-1.1448e+00,  2.1234e-01, -7.5028e-01,  ...,  1.5879e+00,
           1.7561e+00,  2.0116e-01],
         [-3.0622e-01, -1.4202e+00, -1.0124e-01,  ...,  7.9453e-01,
           1.0808e+00, -6.9835e-01],
         [ 6.1804e-04,  1.1306e+00,  1.4697e+00,  ..., -1.8279e+00,
          -1.9963e+00,  2.3231e-01],
         [ 8.5560e-01,  4.4485e-01,  1.9005e-01,  ..., -1.3711e+00,
          -4.6878e-01, -1.9606e-01]],

        [[ 9.4831e-02, -1.2370e+00, -1.4587e+00,  ..., -5.7110e-01,
           4.3495e-01, -3.3620e+00],
         [ 7.3159e-01,  2.2497e-01, -3.1516e-01,  ...,  9.5986e-02,
           5.6782e-01, -1.5481e+00],
         [-2.6487e-01, -1.0557e-01, -7.7925e-01,  ..., -8.7937e-01,
          -1.1653e+00, -5.3163e-01],
         [ 1.5368e+00, -6.0276e-01, -5.1097e-02,  ..., -1.1105e+00,
           9.8018e-01, -2.4366e+00]],

        [[ 1.1116e-01, -2.1593e+00, -1.0023e+00,  ..., -1.7803e+00,
           7.0201e-01, -9.8370e-02],
         [-1.8402e+00,  1.0487e+00,  9.8359e-01,  .

In [10]:
output = block( input )

In [11]:
output.shape

torch.Size([3, 4, 768])

In [12]:
output

tensor([[[-0.9156, -0.2333, -0.9646,  ...,  1.5196,  1.4456,  0.1774],
         [-0.1726, -1.9569,  0.0546,  ...,  0.2374,  0.9705, -0.9097],
         [ 0.0826,  0.9758,  1.5598,  ..., -2.3966, -2.1967, -0.1673],
         [ 0.9158,  0.0209,  0.4494,  ..., -1.2938, -0.6432, -0.2300]],

        [[ 0.0377, -1.9451, -1.6424,  ..., -0.5718,  1.0044, -3.5969],
         [ 1.0341, -0.6554,  0.0725,  ...,  0.2864,  0.8839, -1.4436],
         [-0.0704, -0.5761, -1.1202,  ..., -0.4471, -0.8132, -0.7097],
         [ 1.6478, -1.2046, -0.1360,  ..., -0.8743,  0.9878, -2.0325]],

        [[ 0.1768, -1.9786, -1.4314,  ..., -1.9818,  0.7505,  0.1502],
         [-1.8597,  1.4536,  0.7399,  ...,  0.0807, -1.1772, -0.1912],
         [-2.7976, -0.6707, -0.5596,  ..., -0.2142, -1.4006,  0.9529],
         [ 0.3549, -1.0830, -0.8855,  ...,  0.3625, -0.9275,  0.0281]]],
       grad_fn=<AddBackward0>)